In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import numpy as np
import time
import re

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
aeropuertos = ["CUN","HAVE","ORD","JFK","LAX","BCN","MAD","YYZ","MCO","SFO","LAS","MUC","YUL","MIA","DEN"] 
urls = []
for x in aeropuertos:
    amx_url = f"https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_{x}_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0"
    urls.append(amx_url)
    print(amx_url)

https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_CUN_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_HAVE_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_ORD_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_JFK_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_LAX_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_BCN_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_MAD_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_YYZ_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_MCO_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-m

In [4]:
amx_data = []

for x in urls:
    browser.visit(x)
    time.sleep(15) #aunque esperes mucho tiempo, dependes de la respuesta del servidor de la página
    try:
        browser.find_by_xpath("/html/body/div[2]/div/div/div[1]/div/div[1]/div/div/button").click()
    except:
        print("ok")
    amx_site = browser.html
    amx_soup = bs(amx_site, 'html.parser')
    options = amx_soup.find_all('div', class_="FlightOptionsListItem") #asegurate de que la ventana sea compacta
    print("appending")
    amx_data.append(options)

appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending
ok
appending


In [5]:
precios = []
origenes = []
destinos = []
salidas = []
llegadas = []

for x in amx_data:
    for y in x:
        precio = y.find('div', class_='FlightOptionsFare-price')
        origen = y.find_all('div', class_='FlightOptionsTimeline-city')
        horario = y.find_all('div', class_='FlightOptionsTimeline-time')
        try:
            precios.append(precio.span.text)
            origenes.append(origen[0].text)
            destinos.append(origen[1].text)
            salidas.append(horario[0])
            llegadas.append(horario[1])
        except:
            continue

In [6]:
salidas2 = []

for x in salidas:
    try:
        salidas2.append(x.text)
    except:
        continue

In [7]:
llegadas2 = []

for x in llegadas:
    try:
        llegadas2.append(x.text)
    except:
        continue

In [8]:
df = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas2,
"Hora Llegada": llegadas2,
"Desde": precios 
})

In [9]:
grouped_df = df.groupby(["Hora Salida"])
amx_grouped = grouped_df.max()

In [11]:
amx_grouped.reset_index(inplace=True)
amx_grouped

,Hora Salida,Origen,Destino,Hora Llegada,Desde
0,01:15,MEX,YUL,07:25,$4391
1,06:01,MEX,SFO,19:30,$9471
2,06:30,MEX,SFO,17:28,$9471
3,07:00,MEX,SFO,20:10,$9471
4,07:10,MEX,MIA,11:30,"$10,851"
5,07:15,MEX,JFK,13:00,$9259
6,07:20,MEX,SFO,17:38,$9471
7,07:30,MEX,LAX,22:40,$4824
8,08:00,MEX,MCO,22:40,$4476
9,08:40,MEX,JFK,21:45,$7878


In [12]:
amx_grouped.to_csv("aeromexico.csv")

In [13]:
aeropuertos = ["CUN","HAVE","ORD","JFK","LAX","BCN","MAD","YYZ","MCO","SFO","LAS","MUC","YUL","MIA","DEN"] 
urls = []
for x in aeropuertos:
    amx_url = f"https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd={x}&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web"
    urls.append(amx_url)
    print(amx_url)

https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=CUN&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=HAVE&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=ORD&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=JFK&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=A

In [14]:
avi_prices = []
avi_horarios = []

for x in urls:
    browser.visit(x)
    time.sleep(15) #aunque esperes mucho tiempo, dependes de la respuesta del servidor de la página
    avi_site = browser.html
    avi_soup = bs(avi_site, 'html.parser')
    avi_prices.append(avi_soup.find_all('div', class_='availability-list-fares'))
    avi_horarios.append(avi_soup.find_all('div', class_='flight-details availability-flight-details flight-details-without-button availability-flight-details-without-button row'))
    print("appending")

appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending
appending


In [52]:
precios = []

for x in avi_prices:
    for y in x:
        precios.append(y.find('div', class_='cell-reco-price').span.text)

In [53]:
salidas = []
llegadas = []
origenes = []
destinos = []

for x in avi_horarios:
    for y in x:
        salidas.append(y.find('time', class_='time-from').text)
        llegadas.append(y.find('time', class_='time-to').text)
        origenes.append(y.find('abbr', class_='citycode-from').text.replace("(","").replace(")",""))
        destinos.append(y.find('abbr', class_='citycode-to').text.replace("(","").replace(")",""))

In [54]:
df2 = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora_Salida": salidas,
"Hora_Llegada": llegadas,
"Desde": precios
})

In [59]:
df2

,Origen,Destino,Hora_Salida,Hora_Llegada,Desde
0,MEX,CUN,21:40,11:25,"17,859.00"
1,MEX,ORD,23:20,18:42,"33,526.00"
2,MEX,ORD,18:30,18:42,"21,703.00"
3,MEX,ORD,18:30,18:42,"22,805.00"
4,MEX,JFK,16:50,01:30,"8,176.00"
5,MEX,JFK,23:20,11:50,"36,725.00"
6,MEX,JFK,13:50,02:50,"25,636.00"
7,MEX,JFK,05:00,18:55,"32,069.00"
8,MEX,JFK,04:50,21:00,"7,496.00"
9,MEX,JFK,23:20,18:55,"36,725.00"


In [62]:
grouped_df2 = df2.groupby(["Hora_Salida", "Destino"])
avi_grouped = grouped_df2.max()

In [63]:
avi_grouped

Origen Hora_Llegada      Desde
Hora_Salida Destino                               
04:50       BCN        MEX        12:50  21,279.00
            DEN        MEX        19:45  23,868.00
            JFK        MEX        21:00   7,634.00
            LAS        MEX        21:57  20,090.00
            LAX        MEX        22:45  16,253.00
            MIA        MEX        13:50  16,550.00
            MUC        MEX        16:32  12,456.00
            SFO        MEX        19:05   8,319.00
            YUL        MEX        18:19  20,741.00
            YYZ        MEX        13:50  11,973.00
05:00       BCN        MEX        12:50  49,711.00
            JFK        MEX        21:00  32,069.00
            MAD        MEX        12:25  19,713.00
            MIA        MEX        16:31  25,391.00
            MUC        MEX        16:32  18,461.00
08:20       BCN        MEX        17:00  29,432.00
            MAD        MEX        12:25  28,870.00
            MUC        MEX        17:25  43,281.00
08:35       BCN        MEX        12:50  21,551.00
            LAS        MEX        21:57  26,136.00
            MAD        MEX        12:25  20,837.00
            SFO        MEX        19:05  15,325.00
10:05       BCN        MEX        12:50  21,082.00
            DEN        MEX        10:12  23,868.00
            LAS        MEX        21:57  20,090.00
            LAX        MEX        22:45  18,214.00
            MAD        MEX        12:25  20,571.00
            SFO        MEX        19:05  10,770.00
13:40       MUC        MEX        16:32  35,662.00
13:50       BCN        MEX        17:00  20,508.00
            JFK        MEX        11:50  27,679.00
            MAD        MEX        21:25  19,982.00
            MIA        MEX        10:30  16,549.00
            MUC        MEX        22:40  43,294.00
            YJB        MEX        19:15  20,224.00
            YYZ        MEX        13:50  36,418.00
16:45       LAX        MEX        11:10  20,379.00
            YYZ        MEX        13:50  21,398.00
16:50       DEN        MEX        17:46  35,979.00
            JFK        MEX        01:30   8,176.00
            LAX        MEX        11:10  14,719.00
            MCO        MEX        12:20  24,819.00
            MIA        MEX        13:50  19,115.00
            MUC        MEX        16:32  18,537.00
            SFO        MEX        00:15  12,812.00
            YUL        MEX        17:19  26,937.00
            YYZ        MEX        13:50  17,393.00
18:30       MCO        MEX        12:20  19,367.00
            ORD        MEX        18:42  22,805.00
21:40       CUN        MEX        11:25  17,859.00
            LAX        MEX        18:25  27,852.00
            YYZ        MEX        13:50  26,013.00
23:20       JFK        MEX        21:00  36,725.00
            MAD        MEX        05:00  19,713.00
            MCO        MEX        17:51  24,840.00
            MIA        MEX        16:31  31,109.00
            ORD        MEX        18:42  33,526.00

In [51]:
avi_grouped.reset_index(inplace=True)
avi_grouped

,Hora_Salida,Origen,Destino,Hora_Llegada,Desde
0,04:50,MEX,YYZ,22:45,"8,319.00"
1,05:00,MEX,MUC,21:00,"49,711.00"
2,08:20,MEX,MUC,17:25,"43,281.00"
3,08:35,MEX,SFO,21:57,"26,136.00"
4,10:05,MEX,SFO,22:45,"23,868.00"
5,13:40,MEX,MUC,16:32,"35,662.00"
6,13:50,MEX,YYZ,22:40,"43,294.00"
7,16:45,MEX,YYZ,13:50,"21,398.00"
8,16:50,MEX,YYZ,17:46,"8,176.00"
9,18:30,MEX,ORD,18:42,"22,805.00"


In [29]:
avi_grouped.to_json("avi_json.js", orient="records")

In [22]:
avi_grouped.to_csv("avianca.csv")

In [ ]:
aeropuertos = ["JFK","LAX","MIA"]
urls = []
for x in aeropuertos:
    ua_url = f"https://www.united.com/ual/en/MX/flight-search/book-a-flight/results/?f=MEX&t={x}&d=2019-11-15&tt=1&sc=7&px=1&taxng=1&newHP=True&idx=1"
    urls.append(ua_url)
    print(ua_url)

In [ ]:
ua_data = []

for x in urls:
    browser.visit(x)
    time.sleep(5)
    browser.visit(x)
    time.sleep(15)
    united_site = browser.html
    united_soup = bs(united_site, 'html.parser')
    print("appending")
    ua_data.append(united_soup.find_all('li', class_='flight-block flight-block-fares use-roundtrippricing flight-block-revised'))

In [ ]:
ua_data[2]

In [ ]:
salidas = []
llegadas = []
origenes = []
destinos = []
precios = []

for x in ua_data:
    for y in x:
        xx = y.find('div', class_='flight-time flight-time-depart').text
        xy = y.find('div', class_='flight-time flight-time-arrive').text
        xz = y.find_all('span')
        xw = (y.find('div', class_='price-point price-point-revised use-roundtrippricing').text)
        salidas.append(re.search(r'\d{1,2}(:\d{1,2})', xx).group())
        llegadas.append(re.search(r'\d{1,2}(:\d{1,2})', xy).group())
        origenes.append(xz[1].text)
        destinos.append(xz[3].text)
        precios.append(xw.replace("\n", "").replace("US", "").replace("$",""))

In [ ]:
df3 = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas,
"Hora Llegada": llegadas,
"Desde": precios
})
df3["Origen"] = df3["Origen"].str[-4:]
df3["Destino"] = df3["Destino"].str[-4:]
df3["Origen"] = df3["Origen"].str[:3]
df3["Destino"] = df3["Destino"].str[:3]
df3.drop(37)
df3

In [ ]:
df3.to_csv("united.csv")